### In this notebook, I first generate faces using captions from COCO dataset using ERNIE-ViLG, and then, I using face detector to extract faces from the images generated. 

In [ ]:
#!pip install translators --upgrade

Load caption data

In [ ]:
import paddlehub as hub
import translators as ts
lines = open('face_captions.txt').read().splitlines()

In [ ]:
import random
random.seed(42)
#Generate 1000 random numbers between 0 and 9999
# I plan to generate about 600 images, but may need to come back generate more if some of the images do not contain a good face
randomlist = random.sample(range(0, 9999), 1100)

In [ ]:
len(randomlist)

In [ ]:
randomlist[:10]

Start generating face images

In [ ]:
import paddlehub as hub
import translators as ts
module = hub.Module(name="ernie_vilg")
for i in range(279, 400):
    prompt = ts.baidu(lines[randomlist[i]] + 'showing front face, like a photo in real life', professional_field='common')
    try: # for certain prompts, error message "暂不支持创作该内容，请修改后再试 (does not support this prompt, please modify)" was returned
        module.generate_image(text_prompts=prompt, style='写实风格',  output_dir='./generated_face_images_with_ERNIE-ViLG_version_2/',topk=1)
        print(i)      
    except:
        continue

In [ ]:
i

extract faces from the generated face images 

In [1]:
#set up folders for extracted faces 
# initialize COCO API for instance annotations
import os
dataDir = './'

image_folder = "extracted_generated_faces_version3"

savedirimage = dataDir + image_folder



if not os.path.exists(savedirimage):
	os.makedirs(savedirimage)

In [2]:
# install all packages
!pip install keras
!pip install tensorflow
from keras.models import load_model
!pip install mtcnn
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

import numpy as np
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image
import matplotlib.pyplot as plt


# Simple python package to shut up Tensorflow warnings and logs.
!pip install silence_tensorflow
import silence_tensorflow.auto

2022-11-29 00:49:31.976535: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 00:49:32.194333: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-29 00:49:32.194370: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-29 00:49:33.076277: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

0.1.0


I did not use threshold as filtering criteria because this will gaurantee low FID score later. I still kept the dimension requirement. 

In [3]:
# extract a single face from a given photograph
def extract_face(filename, index, required_size=(250, 250)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    if len(results) >=1:
        x1, y1, width, height = results[0]['box']
        if height-width>=15 and results[0]['confidence'] >= 0.999:
            # bug fix
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            # extract the face
            face = pixels[y1:y2, x1:x2]
            # resize pixels to the model size
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array = np.asarray(image)
            plt.imsave(os.path.join(savedirimage, 'face_img_{}.jpg'.format(index)),face_array)
            return True
    return False

In [4]:
files = os.listdir('generated_face_images_with_ERNIE-ViLG_version_2/')

In [5]:
index =0
for file in files:
    try:
        if extract_face(os.path.join('./generated_face_images_with_ERNIE-ViLG_version_2/',file), index = index):
            print(index)
            index+=1
    except:
        None

2022-11-29 00:49:39.866190: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-29 00:49:39.866347: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-29 00:49:39.866441: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/

1/1 [==============================] - 0s 129ms/step
0
1/1 [==============================] - 0s 124ms/step
1
2/2 [==============================] - 0s 10ms/step
2
1/1 [==============================] - 0s 131ms/step
3
1/1 [==============================] - 0s 132ms/step
4
1/1 [==============================] - 0s 138ms/step
5
1/1 [==============================] - 0s 124ms/step
6
1/1 [==============================] - 0s 133ms/step
7
2/2 [==============================] - 0s 14ms/step
8
1/1 [==============================] - 0s 139ms/step
9
1/1 [==============================] - 0s 143ms/step
10
1/1 [==============================] - 0s 127ms/step
11
2/2 [==============================] - 0s 13ms/step
12
1/1 [==============================] - 0s 125ms/step
13
1/1 [==============================] - 0s 128ms/step
14
1/1 [==============================] - 0s 38ms/step


1/1 [==============================] - 0s 127ms/step
15
1/1 [==============================] - 0s 126ms/step
16
1/1 [==============================] - 0s 129ms/step
17
1/1 [==============================] - 0s 129ms/step
18
1/1 [==============================] - 0s 121ms/step
19
1/1 [==============================] - 0s 149ms/step
20
1/1 [==============================] - 0s 141ms/step
21
1/1 [==============================] - 0s 130ms/step
22
1/1 [==============================] - 0s 126ms/step
23
1/1 [==============================] - 0s 125ms/step
24
1/1 [==============================] - 0s 170ms/step
25
1/1 [==============================] - 0s 142ms/step
26
2/2 [==============================] - 0s 9ms/step
27
1/1 [==============================] - 0s 131ms/step
28
1/1 [==============================] - 0s 121ms/step
29
1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 139ms/step
30
1/1 [==============================] - 1s 501ms/step
31
1/1 [==============================] - 0s 155ms/step
32
1/1 [==============================] - 0s 135ms/step
33
1/1 [==============================] - 0s 133ms/step
34
1/1 [==============================] - 0s 141ms/step
35
1/1 [==============================] - 0s 137ms/step
36
1/1 [==============================] - 0s 122ms/step
37
1/1 [==============================] - 0s 170ms/step
38
1/1 [==============================] - 0s 144ms/step
39
1/1 [==============================] - 0s 136ms/step
40
1/1 [==============================] - 0s 119ms/step
41
1/1 [==============================] - 0s 129ms/step
42
1/1 [==============================] - 0s 138ms/step
43
1/1 [==============================] - 0s 129ms/step
44
1/1 [==============================] - 0s 177ms/step
45
1/1 [==============================] - 0s 144ms/step


1/1 [==============================] - 0s 124ms/step
46
1/1 [==============================] - 0s 140ms/step
47
2/2 [==============================] - 0s 11ms/step
48
1/1 [==============================] - 0s 144ms/step
49
1/1 [==============================] - 0s 145ms/step
50
1/1 [==============================] - 0s 130ms/step
51
1/1 [==============================] - 0s 124ms/step
52
1/1 [==============================] - 0s 151ms/step
53
1/1 [==============================] - 0s 131ms/step
54
1/1 [==============================] - 0s 181ms/step
55
1/1 [==============================] - 0s 138ms/step
56
1/1 [==============================] - 0s 135ms/step
57
1/1 [==============================] - 0s 120ms/step
58
1/1 [==============================] - 0s 30ms/step


...

1/1 [==============================] - 0s 131ms/step
155
1/1 [==============================] - 0s 136ms/step
156
1/1 [==============================] - 0s 181ms/step
157
1/1 [==============================] - 0s 142ms/step
158
1/1 [==============================] - 0s 142ms/step
159
1/1 [==============================] - 0s 143ms/step
160
1/1 [==============================] - 0s 136ms/step
161
1/1 [==============================] - 0s 178ms/step
162
1/1 [==============================] - 0s 136ms/step
163
1/1 [==============================] - 0s 126ms/step
164
1/1 [==============================] - 0s 125ms/step
165
1/1 [==============================] - 0s 171ms/step
166
1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 149ms/step
167
1/1 [==============================] - 0s 131ms/step
168
1/1 [==============================] - 0s 123ms/step
169
1/1 [==============================] - 0s 132ms/step
170
1/1 [==============================] - 0s 138ms/step
171
1/1 [==============================] - 0s 128ms/step
172
1/1 [==============================] - 0s 156ms/step
173
1/1 [==============================] - 0s 156ms/step
174
1/1 [==============================] - 0s 136ms/step
175
1/1 [==============================] - 0s 138ms/step
176
1/1 [==============================] - 0s 129ms/step
177
1/1 [==============================] - 0s 139ms/step
178
1/1 [==============================] - 0s 131ms/step
179
1/1 [==============================] - 0s 165ms/step
180
1/1 [==============================] - 0s 155ms/step
181
3/3 [==============================] - 0s 16ms/step
182
2/2 [==============================] - 0s 10ms/step
183
1/1 [============================

1/1 [==============================] - 0s 139ms/step
185
1/1 [==============================] - 0s 135ms/step
186
1/1 [==============================] - 0s 183ms/step
187
1/1 [==============================] - 0s 130ms/step
188
1/1 [==============================] - 0s 130ms/step
189
1/1 [==============================] - 0s 144ms/step
190
1/1 [==============================] - 0s 128ms/step
191
1/1 [==============================] - 0s 124ms/step
192
1/1 [==============================] - 0s 195ms/step
193
1/1 [==============================] - 0s 167ms/step
194
1/1 [==============================] - 0s 159ms/step
195
1/1 [==============================] - 0s 131ms/step
196
1/1 [==============================] - 0s 128ms/step
197
1/1 [==============================] - 0s 124ms/step
198
1/1 [==============================] - 0s 136ms/step
199
1/1 [==============================] - 0s 148ms/step
200
12/12 [==============================] - 0s 6ms/step


2/2 [==============================] - 0s 16ms/step
201
1/1 [==============================] - 0s 133ms/step
202
1/1 [==============================] - 0s 130ms/step
203
1/1 [==============================] - 0s 135ms/step
204
1/1 [==============================] - 0s 155ms/step
205
1/1 [==============================] - 0s 145ms/step
206
1/1 [==============================] - 0s 163ms/step
207
1/1 [==============================] - 0s 139ms/step
208
1/1 [==============================] - 0s 144ms/step
209
1/1 [==============================] - 0s 143ms/step
210
1/1 [==============================] - 0s 133ms/step
211
1/1 [==============================] - 0s 127ms/step
212
1/1 [==============================] - 1s 578ms/step
213
1/1 [==============================] - 0s 163ms/step
214
1/1 [==============================] - 0s 133ms/step
215
1/1 [==============================] - 0s 130ms/step
216
1/1 [==============================] - 0s 119ms/step
217
1/1 [===========================

1/1 [==============================] - 0s 203ms/step
219
2/2 [==============================] - 0s 9ms/step
220
1/1 [==============================] - 0s 131ms/step
221
1/1 [==============================] - 0s 131ms/step
222
1/1 [==============================] - 0s 122ms/step
223
1/1 [==============================] - 0s 145ms/step
224
1/1 [==============================] - 0s 174ms/step
